## Libraries

In [45]:
import sqlite3
from sqlite3 import Error
import pandas as pd
from pandas import read_excel
import uuid
import re
import math
import time

## Setup Functions

In [46]:
# Connect to SQLite database
def establish_connection(db_file):
    connection = None
    try:
        connection = sqlite3.connect(db_file,timeout=10)
    except Error as e:
        print(e)

    return connection

In [47]:
# Detemine which fields contain data for a given record
def identify_datapoints(data):
    data_fields = {
        'location':False if str(data['HQ_City']) == 'nan' or str(data['HQ_State']) == 'nan' else True,
        'industry':False if str(data['Industry']) == 'nan' else True,
        'provider':False if str(data['Company_Name']) == 'nan' or str(data['URL']) == 'nan' else True,
        'service':False if str(data['Keywords']) == 'nan' else True,
        'executive':False if str(data['Company_Founders']) == 'nan' else True,
        'revenue':False if str(data['Index_Revenues']) == 'nan' else True,
        'employee':False if str(data['Index_Employees']) == 'nan' else True
    }
    
    return data_fields

## Lookup Functions

### Check If Record Exists

In [48]:
# Check if record is already in database
def record_exists(conn,table,params):
    sql = get_lookup_statement(table)
    cur = conn.cursor()
    if(hasMultiple_params(table)):
        cur.execute(sql, params)
    else:
        cur.execute(sql, (params,))
    results = cur.fetchall()
    cur.close()
    if(results == []):
        return False
    else:
        return True

### Get UUID For Existing Records

In [49]:
# Return the UUID for an existing database record
def get_uuid(conn,table,params):
    sql = get_lookup_statement(table).replace('*','uuid')
    cur = conn.cursor()
    if(hasMultiple_params(table)):
        cur.execute(sql, params)
    else:
        cur.execute(sql, (params,))
    result = re.findall("[0-9a-fA-F]{8}\-[0-9a-fA-F]{4}\-[0-9a-fA-F]{4}\-[0-9a-fA-F]{4}\-[0-9a-fA-F]{12}", str(cur.fetchall()))
    cur.close()
    return result[0]

#### Get SQL Statements

In [50]:
# Return the SQL statement to lookup a record in a given table
def get_lookup_statement(table):
    switch = {
        'industry':'SELECT * FROM industry WHERE name = ?',
        'provider':'SELECT * FROM provider WHERE name = ?',
        'location':'SELECT * FROM location WHERE city = ? AND state = ?',
        'service':'SELECT * FROM service WHERE name = ?',
        'executive':'SELECT * FROM executive WHERE first_name = ? AND last_name = ?',
        'industry_service':'SELECT * FROM industry_service WHERE industry_id = ? AND service_id = ?',
        'provider_service':'SELECT * FROM provider_service WHERE provider_id = ? AND service_id = ?',
        'provider_finance':'SELECT * FROM provider_finance WHERE provider_id = ?',
        'revenue':'SELECT * FROM revenue WHERE year = ? AND finance_id = ?',
        'employee':'SELECT * FROM employee WHERE year = ? AND finance_id = ?',
        'funding':'SELECT * FROM funding WHERE finance_id = ?'
    }
    
    return switch.get(table,'Invalid') 

In [51]:
# Determine which tables require multiple parameters for lookup
def hasMultiple_params(table):
    switch = {
        'industry':False,
        'provider':False,
        'location':True,
        'service':False,
        'executive':True,
        'industry_service':True,
        'provider_service':True,
        'provider_finance':False,
        'revenue':True,
        'employee':True,
        'funding':False
    }
    return switch.get(table,'Invalid')

## Insert Functions

In [52]:
# Insert a new record into a database table
def add_record(conn,table,record):
    sql = get_insert_statement(table)
    cur = conn.cursor()
    cur.execute(sql, record)
    cur.close()

#### Get SQL Statements

In [53]:
# Return the SQL statement to insert a record into a given table
def get_insert_statement(table):
    switch = {
        'industry':'INSERT INTO industry(uuid,name) VALUES(?,?);',
        'provider':'INSERT INTO provider(uuid,name,classification,year_founded,phone,website,location_id,industry_id) VALUES(?,?,?,?,?,?,?,?);',
        'location':'INSERT INTO location(uuid,city,state) VALUES(?,?,?);',
        'service':'INSERT INTO service(uuid,name) VALUES(?,?);',
        'executive':'INSERT INTO executive(uuid,first_name,last_name,provider_id) VALUES(?,?,?,?);',
        'industry_service':'INSERT INTO industry_service(uuid,industry_id,service_id) VALUES(?,?,?);',
        'provider_service':'INSERT INTO provider_service(uuid,provider_id,service_id) VALUES(?,?,?);',
        'provider_finance':'INSERT INTO provider_finance(uuid,revenue_growth_rate_1yr,revenue_growth_rate_3yr,employee_growth_rate_1yr,employee_growth_rate_3yr,provider_id) VALUES(?,?,?,?,?,?)',
        'revenue':'INSERT INTO revenue(uuid,year,revenue_total,finance_id) VALUES(?,?,?,?)',
        'employee':'INSERT INTO employee(uuid,year,employee_count,finance_id) VALUES(?,?,?,?)',
        'funding':'INSERT INTO funding(uuid,private_equity,venture_capital,latest_year,valuation,total_funding,finance_id) VALUES (?,?,?,?,?,?,?)'
    }
    return switch.get(table,'Invalid')

## Entity Specific Functions

#### Provider

In [54]:
# Return UUID for a Provider Entity if it exists, otherwise add a new record to the database and return the new UUID
def get_provider_id(conn, record, location_id, industry_id):
    if(record_exists(conn,'provider',record['Company_Name'])):
        return get_uuid(conn,'provider',record['Company_Name'])
    else:
        unique_id = str(uuid.uuid1())
        year = None if math.isnan(record['Year_Founded']) else str(record['Year_Founded'])
        year = year[0:4] if year != None else None
        provider = (unique_id,record['Company_Name'],record['Classification'],year,record['Headquarters_Phone'],record['URL'],location_id,industry_id)
        add_record(conn,'provider',provider)
        return unique_id

#### Industry

In [55]:
# Return UUID for an Industry Entity if it exists, otherwise add a new record to the database and return the new UUID
def get_industry_id(conn,industry):
    if(record_exists(conn,'industry',industry)):
        return get_uuid(conn,'industry',industry)
    else:
        unique_id = str(uuid.uuid1())
        ind = (unique_id,industry)
        add_record(conn,'industry',ind)
        return unique_id

#### Location

In [56]:
# Return UUID for a Location Entity if it exists, otherwise add a new record to the database and return the new UUID
def get_location_id(conn,record):
    location = (record['HQ_City'],record['HQ_State'])
    if(record_exists(conn,'location',location)):
        return get_uuid(conn,'location',location)
    else:
        unique_id = str(uuid.uuid1())
        location = (unique_id,record['HQ_City'],record['HQ_State'])
        add_record(conn,'location',location)
        return unique_id

#### Service

In [57]:
# Return UUIDs for Service Entities if they exist, otherwise add a new records to the database and return the new UUIDs
def get_service_ids(conn,services):
    service_ids = []
    
    for service in services:
        if(record_exists(conn,'service',service)):
            service_ids.append(get_uuid(conn,'service',service))
        else:
            unique_id = str(uuid.uuid1())
            serv = (unique_id,service)
            add_record(conn,'service',serv)
            service_ids.append(unique_id)
    
    return service_ids

#### Executive

In [58]:
# Return UUIDs for Executive Entities if they exist, otherwise add a new records to the database and return the new UUIDs
def get_executive_ids(conn,executives,provider_id):
    executive_ids = []
    for executive in executives:
        name = executive.split()
        name = (name[0],name[-1])
        if(record_exists(conn,'executive',name)):
            executive_ids.append(get_uuid(conn,'executive',name)) 
        else:
            unique_id = str(uuid.uuid1())
            first_name = name[0]
            last_name = name[-1]
            executive = (unique_id,first_name,last_name,provider_id)
            add_record(conn,'executive',executive)
            executive_ids.append(unique_id)
    
    return executive_ids

#### Industry/Service

In [59]:
# Return UUIDs for Industry/Service Entities if they exist, otherwise add a new records to the database and return the new UUIDs
def get_ind_serv_ids(conn,ind_id,serv_ids):
    ind_serv_ids = []
    
    for service in serv_ids:
        ind_serv = (ind_id,service)
        if(record_exists(conn,'industry_service',ind_serv)):
            ind_serv_ids.append(get_uuid(conn,'industry_service',ind_serv)) 
        else:
            unique_id = str(uuid.uuid1())
            industry_service = (unique_id,ind_id,service)
            add_record(conn,'industry_service',industry_service)
            ind_serv_ids.append(unique_id) 
    
    return ind_serv_ids

#### Provider/Service

In [60]:
# Return UUIDs for Provider/Service Entities if they exist, otherwise add a new records to the database and return the new UUIDs
def get_prov_serv_ids(conn,prov_id,serv_ids):
    prov_serv_ids = []
    
    for service in serv_ids:
        prov_serv = (prov_id,service)
        if(record_exists(conn,'provider_service',prov_serv)):
            prov_serv_ids.append(get_uuid(conn,'provider_service',prov_serv)) 
        else:
            unique_id = str(uuid.uuid1())
            provider_service = (unique_id,prov_id,service)
            add_record(conn,'provider_service',provider_service)
            prov_serv_ids.append(unique_id) 
    
    return prov_serv_ids

#### Provider/Finance

In [61]:
# Return UUIDs for Provider/Finance Entities if they exist, otherwise add a new records to the database and return the new UUIDs
def get_prov_fin_id(conn,record,provider_id):
    if(record_exists(conn,'provider_finance',provider_id)):
        return get_uuid(conn,'provider_finance',provider_id)
    else:
        unique_id = str(uuid.uuid1())
        rev_grow_1yr = None if math.isnan(record['Revenues_Growth_Rate_1_Year (%)']) else record['Revenues_Growth_Rate_1_Year (%)']
        rev_grow_3yr = None if math.isnan(record['Revenues_Growth_Rate_3_Year (%)']) else record['Revenues_Growth_Rate_3_Year (%)']
        emp_grow_1yr = None if math.isnan(record['Employees_Growth_Rate_1_Year (%)']) else record['Employees_Growth_Rate_1_Year (%)']
        emp_grow_3yr = None if math.isnan(record['Employees_Growth_Rate_3_Year (%)']) else record['Employees_Growth_Rate_3_Year (%)']
        prov_fin = (unique_id,rev_grow_1yr,rev_grow_3yr,emp_grow_1yr,emp_grow_3yr,provider_id)
        add_record(conn,'provider_finance',prov_fin)
        return unique_id

#### Revenue

In [62]:
# Return UUIDs for Revenue Entities if they exist, otherwise add a new records to the database and return the new UUIDs
def get_revenue_ids(conn,record,finance_id):
    revenues = [x.split(':') for x in str(record).split('|')]
    rev_ids = []
    for val in revenues:
        year = val[0]
        amount = val[1]
        rev = (year,finance_id)
        if(record_exists(conn,'revenue',rev)):
            rev_ids.append(get_uuid(conn,'revenue',rev))
        else:
            unique_id = str(uuid.uuid1())
            rev = (unique_id,year,amount,finance_id)
            add_record(conn,'revenue',rev)
            rev_ids.append(unique_id)
    return rev_ids

#### Employee

In [63]:
# Return UUIDs for Employee Entities if they exist, otherwise add a new records to the database and return the new UUIDs
def get_employee_ids(conn,record,finance_id):
    employees = [x.split(':') for x in str(record).split('|')]
    emp_ids = []
    for val in employees:
        year = val[0]
        amount = val[1]
        emp = (year,finance_id)
        if(record_exists(conn,'employee',emp)):
            emp_ids.append(get_uuid(conn,'employee',emp))
        else:
            unique_id = str(uuid.uuid1())
            emp = (unique_id,year,amount,finance_id)
            add_record(conn,'employee',emp)
            emp_ids.append(unique_id)
    return emp_ids

#### Funding

In [64]:
# Return UUID for a Funding Entity if it exists, otherwise add a new record to the database and return the new UUID
def get_funding_ids(conn,record,finance_id):
    if(record_exists(conn,'funding',finance_id)):
        return get_uuid(conn,'funding',finance_id)
    else:
        unique_id = str(uuid.uuid1())
        pe_backed = record['PE_Backed?']
        vc_backed = record['VC_Backed?']
        val_year = None if math.isnan(record['Latest_Valuation_Year']) else str(record['Latest_Valuation_Year'])
        val_year = val_year[0:4] if val_year != None else None
        val = record['Latest_Valuation']
        total_fund = record['Total_Funding']
        funding = (unique_id, pe_backed, vc_backed, val_year, val, total_fund, finance_id)
        add_record(conn,'funding',funding)
        return unique_id

## Main Functions

In [65]:
# Insert all available datapoints into the database
def insert_data(db_conn,record,hasData):
    location_id = get_location_id(db_conn,record) if hasData['location'] else None
    industry_id = get_industry_id(db_conn,record['Industry']) if hasData['industry'] else None
    provider_id = get_provider_id(db_conn,record,location_id,industry_id) if hasData['provider'] else None
    service_ids = get_service_ids(db_conn,record['Keywords'].split(',')) if hasData['service'] else None
    executive_ids = get_executive_ids(db_conn,record['Company_Founders'].split('|'),provider_id) if hasData['executive'] else None
    
    ind_serv_ids = get_ind_serv_ids(db_conn,industry_id,service_ids) if hasData['industry'] and hasData['service'] else None
    prov_serv_ids = get_prov_serv_ids(db_conn,provider_id,service_ids) if hasData['provider'] and hasData['service'] else None

    finance_id = get_prov_fin_id(db_conn,record,provider_id) if provider_id != None else None
    
    revenue_ids = get_revenue_ids(db_conn,record['Index_Revenues'],finance_id) if hasData['revenue'] and finance_id != None else None
    employee_ids = get_employee_ids(db_conn,record['Index_Employees'],finance_id) if hasData['employee'] and finance_id != None else None
    
    funding_ids = get_funding_ids(db_conn,record,finance_id) if finance_id != None else None
    
    entry = {
        'provider_id':str(provider_id),
        'location_id':str(location_id),
        'industry_id':str(industry_id),
        'service_ids':str(service_ids),
        'executive_ids':str(executive_ids),
        'industry_service_ids':str(ind_serv_ids),
        'provider_service_ids':str(prov_serv_ids),
        'provider_finance_id':str(finance_id),
        'revenue_ids':str(revenue_ids),
        'employee_ids':str(employee_ids),
        'funding_ids':str(funding_ids)
    }
    
    return entry

In [66]:
#  Write UUIDs of added entries to a log file
def write_to_log(entry):
    output_log.write("\n\nAdded entry:")
    for key,item in entry.items():
        output_log.write('\n\t' + key + ' : ' + item)

#### Main

In [68]:
# Iterate over rows from data extract and insert any new records into the database
db_file = r"C:\Users\collinsm4\Documents\Wentworth\Senior\Capstone\msp_prospects.sqlite"
db_conn = establish_connection(db_file)

data_file = r"C:\Users\collinsm4\Documents\Wentworth\Senior\Capstone\Data Extraction\Data.xlsx"
data = pd.read_excel(data_file)

timestamp = time.strftime("%Y.%m.%d_%H.%M.%S",time.localtime())
filename = '\Output_Log_' + timestamp + '.txt'
output_log = open(r"C:\Users\collinsm4\Documents\Wentworth\Senior\Capstone\Data Insert\Log Files" + filename,"w")

for index in range(len(data)):
    record = data.loc[index]    
    available_data = identify_datapoints(record)

    uuids_added = insert_data(db_conn,record,available_data)
    
    write_to_log(uuids_added)    
    

output_log.close()
db_conn.commit()
db_conn.close()